In [2]:
import hashlib
import requests
import sys
import os
import time
from typing import Optional

API_URL = "https://api.pwnedpasswords.com/range/{}"
# Use something meaningful for your environment/team
USER_AGENT = "PasswordAuditScript/1.0 (contact: example@example.com)"
HEADERS = {
    "User-Agent": USER_AGENT,
    # Recommended to reduce correlation via response size
    "Add-Padding": "true",
    # Optional; API returns plain text
    "Accept": "text/plain",
}

def sha1_hex(s: str) -> str:
    """Return uppercase SHA-1 hex digest of a UTF-8 string."""
    return hashlib.sha1(s.encode("utf-8")).hexdigest().upper()

def pwned_count(password: str, session: Optional[requests.Session] = None,
                timeout: float = 10.0, max_retries: int = 3) -> int:
    """
    Return the number of times 'password' appears in breaches (0 if not found).
    Uses the k-anonymity 'range' API. Returns -1 on persistent HTTP/Network error.
    Retries on 429 and other transient non-200 responses.
    """
    digest = sha1_hex(password)
    prefix, suffix = digest[:5], digest[5:]

    # Use provided session (preferred) or a one-off request
    def do_request():
        if session is not None:
            return session.get(API_URL.format(prefix), headers=HEADERS, timeout=timeout)
        return requests.get(API_URL.format(prefix), headers=HEADERS, timeout=timeout)

    backoff = 1.0
    for attempt in range(1, max_retries + 1):
        try:
            resp = do_request()
        except requests.RequestException:
            if attempt == max_retries:
                return -1
            time.sleep(backoff)
            backoff *= 2
            continue

        if resp.status_code == 200:
            # Lines are "HASH_SUFFIX:COUNT"
            for line in resp.text.splitlines():
                if ":" not in line:
                    continue
                hash_suffix, count = line.split(":", 1)
                if hash_suffix.strip() == suffix:
                    try:
                        return int(count.strip())
                    except ValueError:
                        return -1
            return 0

        # Respect Retry-After if present on 429
        if resp.status_code == 429:
            retry_after = resp.headers.get("Retry-After")
            sleep_s = float(retry_after) if retry_after and retry_after.isdigit() else backoff
        else:
            sleep_s = backoff

        if attempt == max_retries:
            return -1
        time.sleep(sleep_s)
        backoff *= 2

    return -1  # Fallback

def check_file_with_pwned_api(filepath: str, throttle_seconds: float = 0.0):
    """
    Read 'username, password' pairs from file and print leak status per user.
    Lines may have spaces; only the FIRST comma splits user/password.
    Ignores blank lines; warns on malformed ones.
    """
    if not os.path.isfile(filepath):
        print(f"❌ File not found: {filepath}")
        return

    print(f"\nChecking passwords from: {filepath}")
    print("Note: Only SHA-1 prefixes are sent to HIBP (k-anonymity).")

    with open(filepath, "r", encoding="utf-8", errors="replace") as f, requests.Session() as sess:
        sess.headers.update(HEADERS)
        for lineno, raw in enumerate(f, start=1):
            line = raw.strip()
            if not line:
                continue
            if "," not in line:
                print(f"Line {lineno}: ⚠️  Skipped (no comma): {line}")
                continue

            username, password = line.split(",", 1)
            username = username.strip()
            password = password.strip()

            if not username:
                print(f"Line {lineno}: ⚠️  Skipped (empty username).")
                continue
            if not password:
                print(f"Line {lineno}: ⚠️  Skipped (empty password) for user '{username}'.")
                continue

            count = pwned_count(password, session=sess)
            if count == -1:
                print(f"{username}: ⚠️  API/network issue (try again later).")
            elif count == 0:
                print(f"{username}: ✅ Not found in known breaches.")
            else:
                print(f"{username}: ⚠️ Found {count} times in breaches — change this password.")

            if throttle_seconds > 0:
                time.sleep(throttle_seconds)

if __name__ == "__main__":
    # Usage: python script.py passwords.txt
    # File format (one per line):
    #   Username1, password1
    #   Username2, password2
    #   Username3,password3
    if len(sys.argv) < 2:
        print("Usage: python script.py <path_to_username_password_file>")
        sys.exit(1)

    filepath = sys.argv[1]
    # For large files, consider polite throttling (e.g., 1.6s) to avoid 429s:
    # check_file_with_pwned_api(filepath, throttle_seconds=1.6)
    check_file_with_pwned_api(filepath)


❌ File not found: -f
